In [1]:
import pandas as pd
import numpy as np


In [76]:
y_true = [1, 0, 0, 1, 1, 0, 1]
y_pred = [1, 0, 1, 1, 0, 0, 1]

In [77]:
# y_true = np.array(y_true)
# y_pred = np.array(y_pred)

# Создание DataFrame для удобства работы с данными
df = pd.DataFrame({'True': y_true, 'Predicted': y_pred})
df

,True,Predicted
0,1,1
1,0,0
2,0,1
3,1,1
4,1,0
5,0,0
6,1,1


In [19]:
df.reset_index()

,index,True,Predicted
0,0,1,1
1,1,0,0
2,2,0,1
3,3,1,1
4,4,1,0
5,5,0,0
6,6,1,1


In [29]:
df.reset_index().groupby(["True", "Predicted"]).count().index.tolist()

[(0, 0), (0, 1), (1, 0), (1, 1)]

RangeIndex(start=0, stop=4, step=1, name=('d', 'd', 'd', 'd'))

In [67]:
np.array(df.groupby(["True", "Predicted"]).value_counts().reset_index().iloc[:,-2:])

array([[0, 2],
       [1, 1],
       [0, 1],
       [1, 3]], dtype=int64)

In [78]:
_, _, fn, tp = np.array(df.groupby(["True", "Predicted"]).value_counts().reset_index().iloc[:, -1])
recall = tp / (tp + fn)
recall

0.75

In [168]:
from typing import List


def recall_at_k(labels: List[int], scores: List[float], k=5) -> float:
    """

    :param labels:
    :param scores:
    :param k:
    :return:
    """
    df = pd.DataFrame({'True': labels, 'Predicted': scores}).iloc[:k]
    print(df)
    tp = df[(df['True'] == 1) & (df['Predicted'] == 1)].shape[0]
    fn = df[(df['True'] == 1) & (df['Predicted'] == 0)].shape[0]
    print(len(df))
    print(tp)
    print(fn)
    try:
        recall = tp / (tp + fn)
    except:
        return 0.0
    return recall

In [158]:
labels = [1, 1, 1, 0, 0, 0]
scores = [0.6, 0.5, 0.4, 0.3, 0.2, 0.1]

In [163]:
recall_at_k(labels, scores, 4)

4
0
0


0.0

In [161]:
import numpy as np
import pandas as pd

def threshold_array(scores, threshold=0.5):
    # Преобразуем массив в pandas Series для удобства работы
    series = pd.Series(scores)

    # Применяем пороговое значение
    binary_array = (series >= threshold).astype(int)

    return binary_array.values

# Пример использования
scores = [0.6, 0.5, 0.4, 0.3, 0.2, 0.1]
thresholded_scores = threshold_array(scores, threshold=0.5)
print(thresholded_scores)

[1 1 0 0 0 0]


In [162]:
y_true = [0, 1, 2, 0, 1, 2]
y_pred = [0, 2, 1, 0, 0, 1]

In [169]:
recall_at_k(y_true, y_pred, 100)

   True  Predicted
0     0          0
1     1          2
2     2          1
3     0          0
4     1          0
5     2          1
6
0
1


0.0

In [189]:
y_true = np.array([1, 0, 1, 0])
y_pred = np.array([1, 0, 1, 1])

In [194]:
def smape(y_true: np.array, y_pred: np.array):

        return np.mean(
            np.where(y_true + y_pred != 0, 2 * np.abs(y_true - y_pred) / (np.abs(y_true) + np.abs(y_pred)), 0))


In [195]:
smape(y_true, y_pred)

C:\Users\Neesty\AppData\Local\Temp\ipykernel_45180\2717382648.py:4: RuntimeWarning: invalid value encountered in divide
  np.where(y_true + y_pred != 0, 2 * np.abs(y_true - y_pred) / (np.abs(y_true) + np.abs(y_pred)), 0))


0.5

In [363]:
def recall_at_k(labels: List[int], scores: List[float], k=5) -> float:
    """

    :param labels:
    :param scores:
    :param k:
    :return:
    """
    # Сортируем оценки в порядке убывания и выбираем топ-k
    top_k_indices = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:k]
    print(f"top_k_indices",top_k_indices)
    non_top_k_labels = [labels[i] for i in range(len(labels)) if i not in top_k_indices]
    print(f"non_top_k_labels",non_top_k_labels)
    # Выбираем метки для топ-k оценок
    top_k_labels = [labels[i] for i in top_k_indices]
    print(f"top_k_labels",top_k_labels)
    # Рассчитываем recall@k
    true_positives = sum(top_k_labels)
    print(f"TP",true_positives)
    # количетво 0 в top_k_labels
    false_positive = len(top_k_labels) - np.sum(top_k_labels)
    print("FP", false_positive)
    true_negatives = len(non_top_k_labels) - sum(non_top_k_labels)
    print("TN", true_negatives)
    false_negatives = sum(non_top_k_labels)
    print("FN", false_negatives)
    total_positives = sum(labels)
    print(f"P",total_positives)
    total_negatives = len(labels) - sum(labels)
    print(f"N",total_negatives)
    #?
    # total_negatives = len(labels) - sum(labels)
    # print(f"total_negatives",total_negatives)

    specificity = true_negatives / (false_positive + true_negatives) if (false_positive + true_negatives) > 0 else 0

    recall = true_positives / total_positives if total_positives > 0 else 0
    precision = true_positives / k if k > 0 else 0


    return (recall, precision, specificity)

In [364]:
labels = [1, 0, 0, 1, 1, 0]
scores = [0.9, 0.4, 0.7, 0.6, 0.5, 0.9]
#[0.3, 0.7, 0.4, 0.3, 0.2, 0.1, 0.4]
#[1, 1, 1, 0, 1, 0, 1]
labels = [1, 0, 0, 1, 1]
scores = [0.8, 0.6, 0.4, 0.7, 0.3]
labels = [0, 0, 0, 0, 0, 1, 1, 1, 1, 1]
scores = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
y_true = np.array(labels)
y_pred = np.array(scores)
# y_pred = np.array([0.3, 0.7, 0.4, 0.3, 0.2, 0.1,0.9, 0.4])
# y_true = np.array([1, 1, 1, 1, 1, 0, 1, 1])
y_pred = np.array([0.3, 0.7, 0.4, 0.3, 0.2, 0.1, 0.9, 0.4])
y_true = np.array([1, 1, 1, 1, 1, 0, 1, 1])

In [365]:
recall_at_k(y_true, y_pred, 5)

top_k_indices [6, 1, 2, 7, 0]
non_top_k_labels [1, 1, 0]
top_k_labels [1, 1, 1, 1, 1]
TP 5
FP 0
TN 1
FN 2
P 7
N 1


(0.7142857142857143, 1.0, 1.0)

In [379]:
import numpy as np

# Исходный массив
arr = np.array([3, 1, 4, 1, 5, 9, 2, 6, 5, 3, 5])

# Сортируем массив
sorted_arr = np.sort(arr, axis=0)[::-1]
print(sorted(arr, reverse=True))
top_k_indices = sorted(range(len(arr)), key=lambda i: arr[i], reverse=True)
print(sorted_arr)
print(top_k_indices)
# Определяем количество элементов в первой части
k = 5

# Делим массив на две части
first_part = sorted_arr[:k]
second_part = sorted_arr[k:]

print("Первая часть:", first_part)
print("Вторая часть:", second_part)


[9, 6, 5, 5, 5, 4, 3, 3, 2, 1, 1]
[9 6 5 5 5 4 3 3 2 1 1]
[5, 7, 4, 8, 10, 2, 0, 9, 6, 1, 3]
Первая часть: [9 6 5 5 5]
Вторая часть: [4 3 3 2 1 1]
